# Installation of requirements

In [1]:
# install requirements

!pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [2]:
# install requirements

!pip install torch-geometric==2.2.0 torch-sparse==0.6.16 torch-scatter==2.1.0 -f https://data.pyg.org/whl/torch-1.12.0+cu113.html

Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 5.4 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773276 sha256=6c4c4783786760693585c83a77cff2f01df41b43c56499a37b4d00ffc6723ee1
  Stored in directory: /root/.cache/pip/wheels/c8/e4/83/5e964867e23f8a61cb8c5d5b9477617b710e96e6ebf1844562
Successfully built torch-geometric


In [3]:
# Connecting the drive with Colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Making sure the working directory is the one

%cd /content/drive/MyDrive/AGILE

/content/drive/MyDrive/AGILE


In [5]:
# install requirements

!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# The following is the content of the file 'config_pretrain.yaml'

batch_size: 512 # batch size
warm_up: 10 # warm-up epochs
epochs: 2 # total number of epochs

load_model: pretrained_gin # resume training
eval_every_n_epochs: 1 # validation frequency
save_every_n_epochs: 5 # automatic model saving frequecy
log_every_n_steps: 50 # print training log frequency

fp16_precision: False # float precision 16 (i.e. True/False)
init_lr: 0.0005 # initial learning rate for Adam
weight_decay: 1e-5 # weight decay for Adam
gpu: cuda:0 # training GPU

model:
  num_layer: 5 # number of graph conv layers
  emb_dim: 300 # embedding dimension in graph conv layers
  feat_dim: 512 # output feature dimention
  drop_ratio: 0 # dropout ratio
  pool: mean # readout pooling (i.e., mean/max/add)

aug: node # molecule graph augmentation strategy (i.e., node/subgraph/mix)
dataset:
  num_workers: 12 # dataloader number of workers
  valid_size: 0.05 # ratio of validation data
  data_path: data/{yourowndata}.csv # path of pre-training data

loss:
  temperature: 0.1 # temperature of NT-Xent loss
  use_cosine_similarity: True # whether to use cosine similarity in NT-Xent loss (i.e. True/False)


# Further ahead from here, I will try to pre-train the MolCLR model with the file 'pretrain.py'

In [ ]:
import os
import shutil
import sys
import torch
import yaml
import numpy as np
from datetime import datetime

In [ ]:
# imports the torch.nn.functional module
# The torch.nn.functional module contains various functions that are commonly used in neural network operations, such as activation functions, loss functions, and other operations commonly applied to tensors.

import torch.nn.functional as F

In [ ]:
# This line imports the SummaryWriter class from the torch.utils.tensorboard module.
# SummaryWriter is a PyTorch utility that enables you to write TensorBoard-compatible logs.
# TensorBoard is a visualization tool provided with TensorFlow, but PyTorch provides integration to use it with PyTorch models as well.

from torch.utils.tensorboard import SummaryWriter

/usr/local/lib/python3.10/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# imports the CosineAnnealingLR class from the torch.optim.lr_scheduler module in PyTorch.
# The learning rate is a hyperparameter that controls how much we are adjusting the weights of our network during training.
# CosineAnnealingLR is a learning rate scheduler in PyTorch that reduces the learning rate following the cosine annealing schedule.
# The learning rate starts at the initial value and is decreased following a cosine function until it reaches a minimum value and then starts increasing again.
# This kind of schedule is often used to improve the convergence and generalization of neural networks during training.

from torch.optim.lr_scheduler import CosineAnnealingLR

In [ ]:
# Changing the working directory to 'utils'

%cd /content/drive/MyDrive/AGILE/utils

/content/drive/MyDrive/AGILE/utils


In [ ]:
#  imports the NTXentLoss class from a custom module named nt_xent inside a package or directory called utils.
# This class likely contains the implementation of the NT-Xent loss function, which is commonly used in contrastive learning tasks.

from utils.nt_xent import NTXentLoss

In [ ]:
# Changing the working directory to 'AGILE'

%cd /content/drive/MyDrive/AGILE

/content/drive/MyDrive/AGILE


In [ ]:
# This code snippet checks if the Apex library is installed and imports it for mixed-precision training if it is available.
# If Apex is not installed, it prints a message indicating that Apex needs to be installed from a specific GitHub repository.

apex_support = False # Initializes the apex_support variable to False.
try: #The code inside the try block attempts to import the amp module from the Apex library.
    sys.path.append("./apex") #  Adds the "./apex" directory to the Python system path, allowing Python to find the Apex module in that directory.
    from apex import amp # Tries to import the amp module from the Apex library.

    apex_support = True # If the import is successful, sets apex_support to True indicating that Apex is available.
except: # If there is an ImportError (i.e., Apex is not installed), the code inside the except block is executed.
    print(
        "Please install apex for mixed precision training from: https://github.com/NVIDIA/apex" # Prints a message indicating that Apex needs to be installed and provides the GitHub repository URL for installation.
    )
    apex_support = False

Please install apex for mixed precision training from: https://github.com/NVIDIA/apex


In [ ]:
# Installing Nvidia apex

import os, sys, shutil
import time
import gc
from contextlib import contextmanager
from pathlib import Path
import random
import numpy as np, pandas as pd
from tqdm import tqdm, tqdm_notebook

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

USE_APEX = True

if USE_APEX:
            with timer('install Nvidia apex'):
                # Installing Nvidia Apex
                os.system('git clone https://github.com/NVIDIA/apex; cd apex; pip install -v --no-cache-dir' +
                          ' --global-option="--cpp_ext" --global-option="--cuda_ext" ./')
                os.system('rm -rf apex/.git') # too many files, Kaggle fails


[install Nvidia apex] done in 1 s


In [ ]:
apex_support = True

In [ ]:
# Changing the working directory to 'apex'

%cd /content/drive/MyDrive/AGILE/apex/apex/amp

/content/drive/MyDrive/AGILE/apex/apex/amp


In [ ]:
from apex import amp


ImportError: ignored